In [1]:
# =============================================================================
# PROJE: PKLot Otopark Doluluk Tespiti
# HAZIRLAYAN: Yunus Emre Edizer
# MODEL: Leaky ReLU + AvgPool (Özel Mimari)
# =============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# --- AYARLAR ---
VERI_YOLU = r"C:\Users\edize\Downloads\2012-09-12"
MODEL_KAYIT_ADI = "Yunus_Emre_Leaky_Model.pth"
# ---------------

# 1. VERİ HAZIRLIĞI
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

print(f"--- VERİ SETİ OKUNUYOR ---\nHedef Klasör: {VERI_YOLU}")

try:
    full_dataset = datasets.ImageFolder(root=VERI_YOLU, transform=transform)
    
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    print(f"Durum: BAŞARILI ✅")
    print(f"Toplam Resim: {len(full_dataset)}")
    print(f"Eğitim Seti: {len(train_dataset)}")
    print(f"Test Seti: {len(test_dataset)}")

except Exception as e:
    print(f"\n!!! HATA !!!: {e}")
    train_loader = None

# 2. MODEL MİMARİSİ (Leaky ReLU + AvgPool)
class LeakyAvgCNN(nn.Module):
    def __init__(self):
        super(LeakyAvgCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        
        # SENİN İMZAN:
        self.avgpool = nn.AvgPool2d(2, 2) 
        self.leaky = nn.LeakyReLU(0.1)
        
        self.fc1 = nn.Linear(32 * 16 * 16, 2)

    def forward(self, x):
        x = self.avgpool(self.leaky(self.conv1(x)))
        x = self.avgpool(self.leaky(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc1(x)
        return x

# 3. EĞİTİM VE TEST DÖNGÜSÜ
if train_loader is not None:
    model = LeakyAvgCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print("\n--- EĞİTİM BAŞLIYOR (Model: LeakyAvgCNN) ---")
    for epoch in range(3):
        correct = 0
        total = 0
        running_loss = 0.0  # Kayıp değerini tutmak için değişken
        
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # İstatistikler
            running_loss += loss.item() # Kaybı topluyoruz
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        # Ortalamaları hesapla
        epoch_acc = 100 * correct / total
        epoch_loss = running_loss / len(train_loader)
        
        # Ekrana hem Doğruluk hem Kayıp yazdırıyoruz
        print(f"Epoch {epoch+1}/3 -> Doğruluk: %{epoch_acc:.2f} | Kayıp (Loss): {epoch_loss:.4f}")
    
    # 4. FİNAL TESTİ
    print("\n--- TEST SONUCU HESAPLANIYOR ---")
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    final_acc = 100 * correct / total
    print(f"SONUÇ: Test Seti Doğruluğu: %{final_acc:.2f}")
    
    torch.save(model.state_dict(), MODEL_KAYIT_ADI)
    print(f"\nModel kaydedildi: {MODEL_KAYIT_ADI}")

--- VERİ SETİ OKUNUYOR ---
Hedef Klasör: C:\Users\edize\Downloads\2012-09-12
Durum: BAŞARILI ✅
Toplam Resim: 8999
Eğitim Seti: 7199
Test Seti: 1800

--- EĞİTİM BAŞLIYOR (Model: LeakyAvgCNN) ---
Epoch 1/3 -> Doğruluk: %94.97 | Kayıp (Loss): 0.1204
Epoch 2/3 -> Doğruluk: %99.90 | Kayıp (Loss): 0.0077
Epoch 3/3 -> Doğruluk: %99.93 | Kayıp (Loss): 0.0037

--- TEST SONUCU HESAPLANIYOR ---
SONUÇ: Test Seti Doğruluğu: %99.94

Model kaydedildi: Yunus_Emre_Leaky_Model.pth
